# Project 2: Subsistence Diets

In [2]:
apikey = "E6haYn1Fyu8cDrLwizdiVB0pPh1FqIlqdEze5lfw" 

In [3]:
# %pip install pandas

import pandas as pd
import numpy as np

### [A] Dietary Reference Intakes Function

Write a function that takes as arguments the characteristics of a person (e.g., age, sex) and returns a `pandas.Series' of Dietary Reference Intakes (DRI's) or "Recommended Daily Allowances" (RDA) of a variety of nutrients appropriate for your population of interest.

In [33]:
rda = pd.read_csv("rda.csv", index_col = 0)

options = ['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 'Female_14_18', 'Male_14_18', 'Female_19_30', 'Male_19_30', 'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']

bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI'])]
bmax = rda.loc[rda['Constraint Type'].isin(['UL'])]

In [34]:
def dietary_ref_intake(age = 20,sex = "Female", data = rda):
    """Takes in age (integer) and sex (string), and returns a Series of dietary reference intakes for the chosen population, you can optionally use a different data frame, the min or max RDAs"""

    if age <= 3:
        col = 'Child_1_3'
    if sex in ["M", "male", "m"]:
        sex = "Male"
    elif sex in ["F", "f", "female"]:
        sex = "Female"
    if age <= 3:
        col = 'Child_1_3'
    elif age >= 51:
        col = sex + "_51U" 
    else:
        age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(51,100)]
        for age_range in age_ranges:
            if age >= age_range[0] and age <= age_range[1]:
                col = sex + '_' + str(age_range[0]) + '_' + str(age_range[1])
    return pd.Series(data[col])  

#### Examples

In [35]:
dietary_ref_intake(age=22,sex='M')

Nutrient
Energy            2400.0
Protein             56.0
Carbohydrate       130.0
Dietary Fiber       33.6
Linoleic Acid       17.0
Linolenic Acid       1.6
Calcium           1000.0
Iron                 8.0
Magnesium          400.0
Phosphorus         700.0
Potassium         4700.0
Sodium            2300.0
Zinc                11.0
Copper               0.9
Selenium            55.0
Vitamin A          900.0
Vitamin E           15.0
Vitamin D           15.0
Vitamin C           90.0
Thiamin              1.2
Riboflavin           1.3
Niacin              16.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            550.0
Vitamin K          120.0
Folate             400.0
Energy            3100.0
Name: Male_19_30, dtype: float64

In [36]:
dietary_ref_intake(age=80,sex='F', data = bmax)

Nutrient
Sodium    2300.0
Energy    3100.0
Name: Female_51U, dtype: float64

### [A] Data on Prices for Different Foods

Construct a google spreadsheet of the prices of different food products for each diet (frozen food diet, meat diet, fresh food diet, liquid diet, and canned-food diet)

In [8]:
# Define file paths again if they are not available
file_paths = {
    "carnivore": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - carnivore_recipes.csv",
    "canned": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - canned_recipes.csv",
    "frozen": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - frozen_recipes.csv",
    "fresh": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - fresh_recipes.csv",
    "liquid": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - liquid_recipes.csv",
    "prices": "~/Documents/GitHub/Project2_EEP153/Wilbur Atwater min_cost_data - prices.csv"
}


# Function to read a dataset
def read_sheet(file_path):
    df = pd.read_csv(file_path, index_col=False)
    df = df.iloc[:, :7].dropna(subset=['parent_foodcode'])
    df = df.reset_index(drop=True)
    return df

# Load prices dataset
prices_df = pd.read_csv(file_paths["prices"])
prices_df['parent_foodcode'] = prices_df['parent_foodcode'].astype(int)  # Convert type for merging

# Function to merge price with a given diet dataset
def read_and_merge_with_prices(diet_name):
    df = read_sheet(file_paths[diet_name])  # Read the diet dataset
    df['parent_foodcode'] = df['parent_foodcode'].astype(int)  # Ensure data type matches for merging
    merged_df = df.merge(prices_df, on="parent_foodcode", how="left")  # Left join to include all diet rows
    return merged_df

# Now run the function without errors
frozen_diet_with_prices = read_and_merge_with_prices("frozen")


In [9]:
#Example of merged diet and price
frozen_diet_with_prices.head()

,parent_foodcode,parent_desc,ingred_code,ingred_desc,ingred_wt,year,mod_code,method,method_description,nhanes,price
0,11460150,"Yogurt, frozen, NS as to flavor, lowfat milk",1298,"Yogurt, frozen, flavors other than chocolate, ...",100.0,2013/2014,NaN,2.0,Links to altEC,Extra,0.335298
1,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8,2011/2012,0.0,1.0,Links to FNDDS,NaN,0.276580
2,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8,2013/2014,NaN,1.0,Links to FNDDS,Extra,0.296941
3,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8,2015/2016,NaN,1.0,Links to FNDDS,Extra,0.301143
4,11460160,"Yogurt, frozen, chocolate, lowfat milk",19166,"Cocoa, dry powder, unsweetened, processed with...",5.2,2011/2012,0.0,1.0,Links to FNDDS,NaN,0.276580


# Liquid Diet Solution

In [ ]:
# %pip install scipy
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='highs')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 14.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
liquid_recipes = pd.read_csv("Wilbur Atwater min_cost_data - liquid_recipes.csv")
nutrition = pd.read_csv("Wilbur Atwater min_cost_data - nutrients.csv")
# from fndds diet problem: normalize weights to percentage terms. 
liquid_recipes['ingred_wt'] = liquid_recipes['ingred_wt']/liquid_recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
liquid_df = liquid_recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(liquid_df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
liquid_df[numeric_cols] = liquid_df[numeric_cols].mul(liquid_df["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. 
#The ** effectively "unpacks" the key value pairs in each dictionary
liquid_df = liquid_df.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "parent_desc": "first"})

liquid_df.index.name = "recipe_id"

food_names = liquid_df["parent_desc"]
print(food_names.head())
liquid_df.head()

recipe_id
353.309772                                   Fruit smoothie, NFS
353.903556            Fruit smoothie, with whole fruit and dairy
382.332989     Fruit smoothie, with whole fruit and dairy, ad...
1258.921519                  Yogurt parfait, low fat, with fruit
1262.662207                        Yogurt, whole milk, baby food
Name: parent_desc, dtype: object


,parent_foodcode,ingred_code,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc,parent_desc
recipe_id,,,,,,,,,,,,,,,,,,,,,
353.309772,353.309772,30.560832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.305814,0.0,0.0,0.0,0.0,0.0,"Fruit smoothie, NFS"
353.903556,353.903556,30.612167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.306328,0.0,0.0,0.0,0.0,0.0,"Fruit smoothie, with whole fruit and dairy"
382.332989,382.332989,33.071245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.330935,0.0,0.0,0.0,0.0,0.0,"Fruit smoothie, with whole fruit and dairy, ad..."
1258.921519,1258.921519,109.913989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.099879,0.0,0.0,0.0,0.0,0.0,"Yogurt parfait, low fat, with fruit"
1262.662207,1262.662207,109.913989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.099879,0.0,0.0,0.0,0.0,0.0,"Yogurt, whole milk, baby food"


In [23]:
prices_df

,year,parent_foodcode,mod_code,method,method_description,nhanes,price
0,2011/2012,11100000,0.0,4,FNDDS recipe with modified SR codes,NaN,0.091897
1,2011/2012,11111000,0.0,1,Links to FNDDS,NaN,0.094505
2,2011/2012,11112110,0.0,1,Links to FNDDS,NaN,0.088018
3,2011/2012,11112210,0.0,1,Links to FNDDS,NaN,0.089206
4,2011/2012,11113000,0.0,1,Links to FNDDS,NaN,0.088424
...,...,...,...,...,...,...,...
15007,2017/2018,95320200,NaN,1,Links to FNDDS,Base,0.120958
15008,2017/2018,95320500,NaN,1,Links to FNDDS,Base,0.091942
15009,2017/2018,95322200,NaN,1,Links to FNDDS,Base,0.114084
15010,2017/2018,95322500,NaN,1,Links to FNDDS,Base,0.089002


In [31]:
prices_liquid = prices_df[["parent_foodcode", "year", "price"]]



prices_liquid = prices_liquid.set_index(["year", "parent_foodcode"])
print(prices_liquid.index.levels[0])

# we'll focus on the latest price data
prices_liquid = prices_liquid.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices_liquid = prices_liquid.dropna(subset="price")
common_recipes = liquid_df.index.intersection(prices_liquid.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 
liquid_df = liquid_df.loc[common_recipes]
prices_liquid = prices_liquid.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices_liquid.index = prices_liquid.index.map(food_names)
A_liquid_all = liquid_df.T


print(f"We have prices for {prices_liquid.shape[0]} unique recipes (FNDDS food codes)")

Index(['2011/2012', '2013/2014', '2015/2016', '2017/2018'], dtype='object', name='year')
We have prices for 381 unique recipes (FNDDS food codes)


In [37]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''
group = "Female_19_30"

# create lower bounds and upper bounds.
bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group]
bmax = rda.loc[rda['Constraint Type'].isin(['UL']), group]

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_liquid_all.reindex(bmin.index).dropna(how='all')
Amax = A_liquid_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices_liquid.shape=}")

bmin.shape=(26,)
Amin.shape=(26, 381)
bmax.shape=(2,)
Amax.shape=(2, 381)
b.shape=(28,)
A.shape=(28, 381)
prices_liquid.shape=(381, 1)


In [41]:
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''

group = 'Female_19_30'
tol = 1e-6

result = lp(prices_liquid, -A, -b, method="highs")
result
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")
diet = pd.Series(result.x,index=prices_liquid.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(round(diet[diet >= tol], 2))

Cost of diet for Female_19_30 is $4.63 per day.

You'll be eating (in 100s of grams or milliliters):
Soy milk, light, chocolate                                                              20.17
Orange juice, 100%,  freshly squeezed                                                    0.72
Vegetable noodle soup, reduced sodium, canned, prepared with water or ready-to-serve     5.72
Corn oil                                                                                 0.75
Molasses                                                                                 0.06
Tea, iced, brewed, black, unsweetened                                                   11.20
Nutritional powder mix, high protein (Herbalife)                                         0.09
dtype: float64


In [42]:
tab = pd.DataFrame({"Outcome":A.to_numpy()@diet.to_numpy(),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)


With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2000.0          2000.0
Protein            58.93592            46.0
Carbohydrate      234.71626           130.0
Dietary Fiber          28.0            28.0
Linoleic Acid      48.96892            12.0
Linolenic Acid     2.092766             1.1
Calcium         2676.872671          1000.0
Iron              18.613537            18.0
Magnesium        476.001765           310.0
Phosphorus      2098.015412           700.0
Potassium            4700.0          4700.0
Zinc               8.368895             8.0
Copper             4.877076             0.9
Selenium               55.0            55.0
Vitamin A       1588.448858           700.0
Vitamin E         18.783996            15.0
Vitamin D         25.281843            15.0
Vitamin C              75.0            75.0
Thiamin            1.342018             1.1
Riboflavin         4.6

In [43]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())


Constraining nutrients are:
['Energy', 'Dietary Fiber', 'Potassium', 'Selenium', 'Vitamin C', 'Folate']
